In [118]:
from utils.omfhandlers import BlockModelHandler
import example as gltf
import utils.omf as omf
import numpy as np
project = omf.load('../../assets/v0/test_file.omf')

In [119]:
t = project
bm = t.elements[-1]

In [120]:
bm.serialize()
handled_bm = BlockModelHandler(bm)
bm_df = handled_bm.get_bm_dataframe
bm_df = bm_df[bm_df['CU_pct'] >= 2.4]


In [121]:
def normalize_vector(vector):
    norm = np.linalg.norm(vector)
    return vector / norm if norm != 0 else vector

In [122]:
def set_color(grade):
    if grade < 2.5:
        color = (0, 0, 1, 1)
    elif grade < 3:
        color = (0, 0.5, 1, 1)
    elif grade < 3.5:
        color = (0, 1, 0.5, 1)
    elif grade < 4:
        color = (1, 1, 0, 1)
    elif grade < 4.5:
        color = (1, 0.5, 0, 1)
    else:
        color = (1, 0, 0, 1)
    return color

Option 1 - common verticies

In [123]:
def block_model_com(x_size, y_size, z_size, x, y, z, indices_offset, grade):
    vertex_data = np.zeros(8, dtype=[
        ("position", np.float32, 3),
        ("normal", np.float32, 3),
        ("color", np.float32, 4),
    ])
    
    
    vertex_data["position"] = [
        (x, y, z),  # 0 Bottom-front-left
        (x + x_size, y, z),  # 1 Bottom-front-right
        (x, y + y_size, z),  # 2 Bottom-back-left
        (x + x_size, y + y_size, z),  # 3 Bottom-back-right
        (x, y, z + z_size),  # 4 Top-front-left
        (x + x_size, y, z + z_size),  # 5 Top-front-right
        (x, y + y_size, z + z_size),  # 6 Top-back-left
        (x + x_size, y + y_size, z + z_size),  # 7 Top-back-right
    ]

    vertex_data["normal"] = [
        (-1, -1, 1),  # 0 Bottom-front-left (average of bottom, front, left face normals)
        (1, -1, 1),   # 1 Bottom-front-right (average of bottom, front, right face normals)
        (-1, 1, 1),   # 2 Bottom-back-left (average of bottom, back, left face normals)
        (1, 1, 1),    # 3 Bottom-back-right (average of bottom, back, right face normals)
        (-1, -1, -1), # 4 Top-front-left (average of top, front, left face normals)
        (1, -1, -1),  # 5 Top-front-right (average of top, front, right face normals)
        (-1, 1, -1),  # 6 Top-back-left (average of top, back, left face normals)
        (1, 1, -1),   # 7 Top-back-right (average of top, back, right face normals)
    ]

    # Normalize the normals
    vertex_data["normal"] = [normalize_vector(np.array(normal)) for normal in vertex_data["normal"]]
    # vertex_data["texCoord0"] = [(0, 0), (0, +1), (+1, 0), (+1, +1)]
    # vertex_data["color"] = [(1, 0, 0, 1), (0, 1, 0, 1), (0, 0, 1, 1), (1, 1, 0, 1)]
    
    index_data = np.array([
        # Front face
        0, 1, 2,  # First triangle (bottom-left, bottom-right, top-left)
        2, 1, 3,  # Second triangle (top-left, bottom-right, top-right)
    
        # Back face
        5, 4, 7,  # First triangle (bottom-right, bottom-left, top-right)
        7, 4, 6,  # Second triangle (top-right, bottom-left, top-left)
    
        # Top face
        2, 3, 6,  # First triangle (top-back-left, top-back-right, top-front-left)
        6, 3, 7,  # Second triangle (top-front-left, top-back-right, top-front-right)
    
        # Bottom face
        1, 0, 5,  # First triangle (bottom-back-right, bottom-back-left, bottom-front-right)
        5, 0, 4,  # Second triangle (bottom-front-right, bottom-back-left, bottom-front-left)
    
        # Left face
        4, 0, 6,  # First triangle (bottom-front-left, bottom-back-left, top-front-left)
        6, 0, 2,  # Second triangle (top-front-left, bottom-back-left, top-back-left)
    
        # Right face
        1, 5, 3,  # First triangle (bottom-back-right, bottom-front-right, top-back-right)
        3, 5, 7,  # Second triangle (top-back-right, bottom-front-right, top-front-right)
    ], dtype=np.uint16)+ indices_offset * 8

    
    color = set_color(grade)
    vertex_data["color"] = [color] * 8


    return vertex_data, index_data

Option 2 Individual verticies

In [124]:
def block_model_ind(x_size, y_size, z_size, x, y, z, indices_offset, grade):
    vertex_data = np.zeros(24, dtype=[
        ("position", np.float32, 3),
        ("normal", np.float32, 3),
        ("color", np.float32, 4),
    ])
    # v6----------v5
    # /|          /|
    # v1---------v0|
    # | |        | |
    # | |v7------|-|v4
    # |/         |/
    # v2---------v3
    vertex_data["position"] = [
        # v0-v1-v2-v3 front
        (x + x_size, y, z + z_size), (x, y, z + z_size), (x, y, z), (x + x_size, y, z),
        # v0-v3-v4-v5 right
        (x + x_size, y, z + z_size), (x + x_size, y, z), (x + x_size, y + y_size, z), (x + x_size, y + y_size, z + z_size), 
        # v0-v5-v6-v1 up
        (x + x_size, y, z + z_size), (x + x_size, y + y_size, z + z_size), (x, y + y_size, z + z_size), (x, y, z + z_size),
        # v1-v6-v7-v2 left
        (x, y, z + z_size), (x, y + y_size, z + z_size), (x, y + y_size, z), (x, y, z),
        # v7-v4-v3-v2 down
        (x, y + y_size, z), (x + x_size, y + y_size, z), (x + x_size, y, z), (x, y, z),
        # v4-v7-v6-v5 back
        (x + x_size, y + y_size, z), (x, y + y_size, z), (x, y + y_size, z + z_size), (x + x_size, y + y_size, z + z_size),
    ]

    vertex_data["normal"] = [
                (0.0, 0.0, 1.0), (0.0, 0.0, 1.0), (0.0, 0.0, 1.0), (0.0, 0.0, 1.0), # v0-v1-v2-v3 front
                (1.0, 0.0, 0.0), (1.0, 0.0, 0.0), (1.0, 0.0, 0.0), (1.0, 0.0, 0.0), # v0-v3-v4-v5 right
                (0.0, 1.0, 0.0), (0.0, 1.0, 0.0), (0.0, 1.0, 0.0), (0.0, 1.0, 0.0), # v0-v5-v6-v1 up
                (-1.0, 0.0, 0.0), (-1.0, 0.0, 0.0), (-1.0, 0.0, 0.0), (-1.0, 0.0, 0.0), # v1-v6-v7-v2 left
                (0.0, -1.0, 0.0), (0.0, -1.0, 0.0), (0.0, -1.0, 0.0), (0.0, -1.0, 0.0),# v7-v4-v3-v2 down
                (0.0, 0.0, -1.0), (0.0, 0.0, -1.0), (0.0, 0.0, -1.0), (0.0, 0.0, -1.0) # v4-v7-v6-v5 back  
            ]
    vertex_data["normal"] = [normalize_vector(np.array(normal)) for normal in vertex_data["normal"]]
    
    index_data = np.array([
        0, 1, 2, 0, 2, 3,       # front
        4, 5, 6, 4, 6, 7,       # right
        8, 9, 10, 8, 10, 11,    # up
        12, 13, 14, 12, 14, 15, # left
        16, 17, 18, 16, 18, 19, # down
        20, 21, 22, 20, 22, 23  # back 
    ], dtype=np.uint16) + indices_offset * 24
    color = set_color(grade)
    vertex_data["color"] = [color] * 24


    return vertex_data, index_data

In [125]:
vertex_data_list = []
index_data_list = []
indices_offset = 0

for index, block in bm_df.iterrows():
    vertex_data, index_data = block_model_ind(block['x_size'], block['y_size'], block['z_size'], block['x_coord'], block['y_coord'], block['z_coord'], indices_offset, block['CU_pct'])
    indices_offset += 1

    vertex_data_list.append(vertex_data)
    index_data_list.append(index_data)

# Concatenate data
final_vertex_data = np.concatenate(vertex_data_list)
final_index_data = np.concatenate(index_data_list)

In [126]:
gltf_path = "../../assets/gltf/block_model_individual.gltf"
bin_path = "../../assets/gltf/block_model_individual.bin"

document, buffers = gltf.numpy_to_gltf(final_vertex_data, final_index_data, gltf_path, bin_path)

gltf.save(gltf_path, bin_path, document, buffers)
buffers

[array([([445220., 494060.,   2670.], [ 0.,  0.,  1.], [0., 0., 1., 1.]),
        ([445210., 494060.,   2670.], [ 0.,  0.,  1.], [0., 0., 1., 1.]),
        ([445210., 494060.,   2660.], [ 0.,  0.,  1.], [0., 0., 1., 1.]),
        ...,
        ([445620., 493580.,   2770.], [ 0.,  0., -1.], [0., 0., 1., 1.]),
        ([445620., 493580.,   2780.], [ 0.,  0., -1.], [0., 0., 1., 1.]),
        ([445630., 493580.,   2780.], [ 0.,  0., -1.], [0., 0., 1., 1.])],
       dtype=[('position', '<f4', (3,)), ('normal', '<f4', (3,)), ('color', '<f4', (4,))]),
 array([     0,      1,      2, ..., 221540, 221542, 221543], dtype=uint32)]